In [1]:
''' Jupyter Notebook is an incredibly powerful tool for interactively developing and presenting data science projects.  '''
''' It allows you to run code, display the results, and add explanations, formulas, and charts all in one place.  '''
''' This job is used to process data for both WMx/OMx Elastic (ES) pipeline queue’s for uploading of data into ES. '''
# ** Prometheus Monitoring DB Test ** 
print(f"** Data Pipeline Status **")

** Data Pipeline Status **


In [4]:
import logging
import os
import sys
import json
from dotenv import load_dotenv
from IPython.display import HTML
import pandas as pd
import dotenv
import datetime
import requests

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

In [ ]:
logger = logging.getLogger("db-api-call")

In [ ]:
''' pip install python-dotenv'''
# load_dotenv() # will search for .env file in local folder and load variables 
# Reload the variables from your .env file, overriding existing ones
dotenv.load_dotenv(".env", override=True)

In [ ]:
def get_time_difference(DB_NAME, audit_process_name_time):
        ''' get time difference'''
        audit_process_name_time = datetime.datetime.strptime(audit_process_name_time, "%Y-%m-%d %H:%M:%S")

        # lock.acquire()
        ''' test has CDT time '''
        if str(DB_NAME).upper() == 'test':
            now_time = datetime.datetime.now() - datetime.timedelta(hours=1)
        else:
            now_time = datetime.datetime.now()

        # print(f"audit_process_name_time - {audit_process_name_time}, : {type(audit_process_name_time)}, now_time - {now_time} : {type(now_time)}")
        date_diff = now_time-audit_process_name_time
        # date_diffs = (now_time-audit_process_name_time)
        # date_diff = audit_process_name_time-now_time
        # print(f"Time Difference - {date_diff}")
        time_hours = date_diff.seconds / 3600
        # print(f"Time Difference to hours - {time_hours}")
        # lock.release()

        if now_time < audit_process_name_time:
            # logging.info(f"now_time < audit_process_name_time - {round(-time_hours, 2)}")
            if time_hours == 0:
                return 0
            return round(-time_hours, 2)
        else:
            # logging.info(f"now_time > audit_process_name_time - {round(time_hours, 2)}")
            return round(time_hours, 2)

In [ ]:
''' call to DB interface RestAPI'''
pd_colmuns, pd_values = [], []
pd_dataframe_dict = {}
def get_db_active(env, sql):
    db_url = os.getenv(str(env).upper())
    request_body = {
        "db_url" : db_url,
        "sql" : sql
    }
    
    # logging.info("db_http_host : {}, db_url : {}, sql : {}".format(os.getenv("DB_API_HOST"), db_url, sql))
    http_urls = "http://{}/db/get_db_query".format(os.getenv("DB_API_HOST"))
    resp = requests.post(url=http_urls, json=request_body, timeout=600)
    # logging.info(f"resp status code : {resp.status_code}")

    ''' resp body '''
    response = resp.json()
    # logging.info(json.dumps(resp.json(), indent=2))
    # logging.info(type(response))

    if resp.status_code != 200:
        return None

    '''
    for row in response['results']:
        for k, v in row.items():
            if k not in pd_dataframe_dict.keys():
                pd_dataframe_dict.update({k : [str(v)]})
            else:
                pd_dataframe_dict.update({k : pd_dataframe_dict.get(k) + [v]})

    # logging.info(json.dumps(pd_dataframe_dict, indent=2))
    return pd_dataframe_dict
    '''
    # return response['results']        
    # return response
    ''' 
    data = {
        'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40]
    }
     
    df = pd.DataFrame(data)
           Name  Age
    0     Alice   25
    1       Bob   30
    2   Charlie   35
    3     David   40
    '''
    results_records_dict = {}
    processname, status, addts, processed_recs, dbid = [],[],[],[],[]
    loop = 0
    recent_data_pipeline_processed = -1
    for each_row in response['results']:
        if loop < 1:
                recent_data_pipeline_processed = each_row.get("ADDTS")
        processname.append(each_row.get("PROCESSNAME"))
        status.append(each_row.get("STATUS"))
        addts.append(each_row.get("ADDTS"))
        processed_recs.append("{:,}".format(each_row.get("COUNT(*)")))
        dbid.append(each_row.get("DBID"))
        loop += 1

    ''' around 30 mintues '''
    ''' Subtract the current timestamp and the max_addts to determine ''' 
    time_threshold =  0.6
    data_pipeline_result = [''] * len(processname)
    DATA_PIPELINE_PRCESSED_WORK = 'FAIL'
    CHAR_SPLIT = "_"

    if CHAR_SPLIT in env:    
            time_gap_recent_data_pipeline_processed = get_time_difference(env.split(CHAR_SPLIT)[0], recent_data_pipeline_processed)
            # print(f"recent_data_pipeline_processed : {recent_data_pipeline_processed}, time_gap_recent_data_pipeline_processed : {time_gap_recent_data_pipeline_processed}")
            if time_gap_recent_data_pipeline_processed < time_threshold:
                    DATA_PIPELINE_PRCESSED_WORK = 'OK'
                    data_pipeline_result[0] = DATA_PIPELINE_PRCESSED_WORK
            else:
                    data_pipeline_result[0] = 'FAIL'
                    
    results_records_dict.update({
            'PROCESSNAME' : processname, 
            'STATUS' : status, 
            'ADDTS' : addts, 
            'COUNT(*)' : processed_recs, 
            'DBID' : dbid,
            'DATA_PIPELINE_WORK' : data_pipeline_result
         })

    return results_records_dict

In [ ]:
def get_db_work(env):
    ''' two db's'''
    try:
        pd_dataframe_dict = get_db_active(f"{env}_A", os.getenv("SQL"))
        df = pd.DataFrame(pd_dataframe_dict)
        display(df)
        
        pd_dataframe_dict = get_db_active(f"{env}_B", os.getenv("SQL"))
        df = pd.DataFrame(pd_dataframe_dict)
        # print(df.head(10))
        display(df)
       
    except Exception as e:
        logger.error(e)

    # finally:
    #     logger.info("Job is being performed correctly")

In [ ]:
''' Validate if data is processing'''
pd_dataframe_dict = get_db_work("local")

In [5]:
# HTML('<pre style="background-color: #A52A2A; color: white;">Job is being performed correctly</pre>')